In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury
from datetime import timedelta
import warnings
import duckdb
import os
import requests

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST

# # CONTROL DATE TO GO BACK AND RELOAD HISTORICAL DATA
# df = df[(df.Date != now)]

Today's date: 2025-12-26


In [3]:
%run ./common_utils.ipynb

In [4]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms['Date'] = pd.to_datetime(df_gms.Date)
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
df_gms['gm_id'] = df_gms.AwayABV + "_" + df_gms.HomeABV
df_gms['gm_id2'] = df_gms.HomeABV + "_" + df_gms.AwayABV
gms_today = df_gms.gm_id.tolist() + df_gms.gm_id2.tolist()
df_gms = df_gms.drop(['gm_id', 'gm_id2'], axis=1)
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,AwayPTS,HomeTeam,HomeABV,HomePTS,Arena,AwayB2B,HomeB2B,cup_gm,pstszn_gm,rtrvd
449,2025-12-26,7:00p,Miami Heat,MIA,NaN,Atlanta Hawks,ATL,NaN,State Farm Arena,0,0,0,0,0
450,2025-12-26,7:00p,Boston Celtics,BOS,NaN,Indiana Pacers,IND,NaN,Gainbridge Fieldhouse,0,0,0,0,0
451,2025-12-26,7:00p,Charlotte Hornets,CHO,NaN,Orlando Magic,ORL,NaN,Kia Center,0,0,0,0,0
452,2025-12-26,7:00p,Toronto Raptors,TOR,NaN,Washington Wizards,WAS,NaN,Capital One Arena,0,0,0,0,0
453,2025-12-26,7:30p,Philadelphia 76ers,PHI,NaN,Chicago Bulls,CHI,NaN,United Center,0,0,0,0,0
454,2025-12-26,8:00p,Milwaukee Bucks,MIL,NaN,Memphis Grizzlies,MEM,NaN,FedExForum,0,0,0,0,0
455,2025-12-26,8:00p,Phoenix Suns,PHO,NaN,New Orleans Pelicans,NOP,NaN,Smoothie King Center,0,0,0,0,0
456,2025-12-26,9:30p,Detroit Pistons,DET,NaN,Utah Jazz,UTA,NaN,Delta Center,0,0,0,0,0
457,2025-12-26,10:00p,Los Angeles Clippers,LAC,NaN,Portland Trail Blazers,POR,NaN,Moda Center,0,0,0,0,0


In [97]:
year=2025

In [99]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status != 'Available'""").fetchdf()
partition_save_df(df_inj, f"../tables/{year}/injuries.csv")

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = {year} AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              WHERE Status = 'Out'
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-12-26_07_15PM.
../tables/2025/injuries.csv saved!


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,MIL,Giannis Antetokounmpo,PF,32.470588,1,2025-12-26,Out,Injury/Illness - Right Calf; Strain
1,UTA,Walker Kessler,C,25.200000,7,2025-12-26,Out,Injury/Illness - Left Shoulder; Injury Recovery
2,POR,Jrue Holiday,PG,24.125000,12,2025-12-26,Out,Injury/Illness - Right Calf; Strain
3,MIA,Bam Adebayo,C,23.916667,9,2025-12-26,Out,Injury/Illness - Lower Back; Soreness
4,ORL,Franz Wagner,SF,23.145833,10,2025-12-26,Out,Injury/Illness - Left High Ankle; Sprain
5,ATL,Kristaps Porzingis,C,22.500000,12,2025-12-26,Out,Injury/Illness - N/A; Illness
6,MEM,Zach Edey,C,22.227273,15,2025-12-26,Out,Injury/Illness - Left Ankle; Stress Reaction
7,MIA,Tyler Herro,SG,22.000000,6,2025-12-26,Out,Injury/Illness - Right Big Toe; Contusion
8,TOR,RJ Barrett,SF,19.764706,18,2025-12-26,Out,Injury/Illness - Right Knee; Sprain
9,ORL,Jalen Suggs,PG,19.333333,22,2025-12-26,Out,Injury/Illness - Left Hip; Contusion


In [6]:
def get_game_odds():

    dk_tm_mapping = {
                    "ATL Hawks": "ATL", "BKN Nets": "BRK", "BOS Celtics": "BOS", "CHA Hornets": "CHO", "CHI Bulls": "CHI",
                    "CLE Cavaliers": "CLE", "DAL Mavericks": "DAL", "DEN Nuggets": "DEN", "DET Pistons": "DET", 
                    "GS Warriors": "GSW", "HOU Rockets": "HOU", "IND Pacers": "IND", "LA Clippers": "LAC", 
                    "LA Lakers": "LAL", "MEM Grizzlies": "MEM", "MIA Heat": "MIA", "MIL Bucks": "MIL", "MIN Timberwolves": "MIN",
                    "NO Pelicans": "NOP", "NY Knicks": "NYK", "OKC Thunder": "OKC", "ORL Magic": "ORL", "PHI 76ers": "PHI",
                    "PHO Suns": "PHO", "POR Trail Blazers": "POR", "SA Spurs": "SAS", "SAC Kings": "SAC", "TOR Raptors": "TOR",
                    "UTA Jazz": "UTA", "WAS Wizards": "WAS", "GSW Warriors": "GSW", "LAL Lakers": "LAL", "NOP Pelicans": "NOP",
                    "NYK Knicks": "NYK", "PHX Suns": "PHO", "SAS Spurs": "SAS"
                    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }
    response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C4511&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%274511%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%274511%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
    if response.status_code != 200:
        raise Exception('Bad Request')

    ids = []
    rows = []
    for i in range(len(response.json()['selections'])):
        if 'HC' in response.json()['selections'][i]['id'] or 'OU' in response.json()['selections'][i]['id']:
            ids.append(response.json()['selections'][i])
    for i in range(0, len(ids) - 1, 4):
        team1 = ids[i]['label']
        spread1 = ids[i]['points']
        team2 = ids[i+1]['label']
        spread2 = ids[i+1]['points']
        total = ids[i+2]['points']
        rows.append({"Team": team1, "Opp": team2, "Spread": spread1, "Total": total})
        rows.append({"Team": team2, "Opp": team1, "Spread": spread2, "Total": total})
    df_tm_bets = pd.DataFrame(rows)
    
    for key, value in dk_tm_mapping.items():
        df_tm_bets['Team'] = np.where(df_tm_bets.Team == key, value, df_tm_bets.Team)
        df_tm_bets['Opp'] = np.where(df_tm_bets.Opp == key, value, df_tm_bets.Opp)

    df_tm_bets['gm_id'] = df_tm_bets.Team + "_" + df_tm_bets.Opp
    df_tm_bets = df_tm_bets[(df_tm_bets.gm_id.isin(gms_today))].drop(['Opp', 'gm_id'], axis=1)
    
    return df_tm_bets

def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": 13508, "BLK": 13780, "STL_BLK": 13781, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines = pd.merge(df_lines, get_game_odds(), on='Team')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

PermissionError: [Errno 13] Permission denied: '../tables/2025/parlay_lines.csv'

In [27]:
def pick_finder(stat, collect=False):
    df_actuals = pd.read_csv(f"../tables/{year}/parlay_actuals.csv")
    
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = {year} AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = {year} AND {stat} > 0 AND Pos != 'None'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = {year} AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 27 AND Season = {year} AND Pos = '{pos}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()

        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        
        # Add hit odds (df_actuals)
        df_hit_odds = con.execute(f"""WITH overs AS 
                                        (SELECT Team, Player, SUM({stat}_Diff) as O_Diff, count(*) as Hits FROM df_actuals 
                                        WHERE {stat}_Result = 'O' 
                                        GROUP BY Team, Player)

                                        SELECT overs.Team, overs.Player, Hits, Misses,
                                        O_Diff + U_Diff as {stat}_P_Diff,
                                        hits / (misses + hits) AS {stat}_Hit_Pct, misses / (misses + hits) AS Miss_Pct FROM

                                       (SELECT Team, Player, SUM({stat}_Diff) as U_Diff, count(*) as Misses FROM df_actuals 
                                       WHERE {stat}_Result = 'U' AND {stat}_line IS NOT NULL
                                       GROUP BY Team, Player) unders

                                       JOIN overs ON overs.Team = unders.Team AND overs.Player = unders.Player
                                       ORDER BY {stat}_Hit_Pct DESC""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.*, df_hit_odds.{stat}_Hit_Pct, df_hit_odds.{stat}_P_Diff FROM df_picks LEFT JOIN df_hit_odds 
                                   ON df_picks.Team = df_hit_odds.Team AND df_picks.Player = df_hit_odds.Player""").fetchdf()     
        
        # Add Spread and Total
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE({stat}_Hit_Pct, {stat}_P_Diff), df_lines.{stat}_line, {stat}_Hit_Pct, {stat}_P_Diff, 
                                   Spread, Total FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player
                                   WHERE {stat}_line IS NOT NULL
                                   ORDER BY Def_Rk DESC""").fetchdf()
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [13]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

===PTS===
PG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk
0,SAS,0,Jordan McLaughlin,4.666667,5.2,2.25,4,NYK,17.947368,17.4,2
1,NYK,0,Jalen Brunson,28.826087,32.8,NaN,1,SAS,21.476190,21.0,19
2,NYK,0,Tyler Kolek,4.470588,4.2,NaN,5,SAS,21.476190,21.0,19
3,SAS,0,De'Aaron Fox,23.882353,22.4,NaN,2,NYK,17.947368,17.4,2
4,SAS,0,Stephon Castle,18.437500,17.4,NaN,3,NYK,17.947368,17.4,2


SG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk
0,SAS,0,Devin Vassell,15.720000,15.8,12.00,1,NYK,19.227273,17.0,28
1,SAS,0,Lindy Waters III,3.125000,2.6,0.00,6,NYK,19.227273,17.0,28
2,NYK,0,Jordan Clarkson,10.000000,8.0,16.25,5,SAS,18.720000,22.0,25
3,SAS,0,Dylan Harper,13.066667,12.4,NaN,2,NYK,19.227273,17.0,28
4,NYK,0,Pacome Dadiet,2.000000,2.0,NaN,7,SAS,18.720000,22.0,25


SF


...

,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk
0,NYK,0,Mikal Bridges,3.409091,2.2,1.75,1,SAS,1.466667,2.2,7
1,NYK,0,Josh Hart,2.157895,2.4,2.25,2,SAS,1.466667,2.2,7
2,SAS,0,Julian Champagnie,1.800000,1.8,0.25,3,NYK,1.050000,1.0,2
3,SAS,0,Keldon Johnson,1.375000,1.6,0.00,4,NYK,1.050000,1.0,2
4,NYK,0,Mohamed Diawara,1.000000,1.0,NaN,5,SAS,1.466667,2.2,7


PF


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk
0,SAS,0,Harrison Barnes,1.631579,1.2,1.00,2,NYK,1.5000,1.6,7
1,SAS,0,Jeremy Sochan,1.500000,1.4,0.25,3,NYK,1.5000,1.6,7
2,NYK,0,OG Anunoby,3.000000,3.2,NaN,1,SAS,1.4375,1.2,5
3,SAS,0,Carter Bryant,1.200000,1.2,NaN,4,NYK,1.5000,1.6,7


C


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk
0,NYK,0,Karl-Anthony Towns,1.950000,1.800000,1.000000,4,SAS,2.071429,2.4,13
1,SAS,0,Luke Kornet,2.230769,2.800000,1.533333,3,NYK,1.823529,2.4,7
2,SAS,0,Kelly Olynyk,1.500000,1.500000,0.428571,6,NYK,1.823529,2.4,7
3,NYK,0,Mitchell Robinson,2.583333,2.600000,NaN,2,SAS,2.071429,2.4,13
4,NYK,0,Ariel Hukporti,1.666667,1.800000,NaN,5,SAS,2.071429,2.4,13
5,SAS,0,Victor Wembanyama,4.538462,3.600000,NaN,1,NYK,1.823529,2.4,7
6,NYK,0,Guerschon Yabusele,1.000000,1.000000,NaN,8,SAS,2.071429,2.4,13
7,SAS,0,Bismack Biyombo,1.333333,1.333333,NaN,7,NYK,1.823529,2.4,7


In [28]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Spread, Total, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk, Spread, Total) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-12-16
../tables/2025/parlay_stats.csv saved!


,Date,Team,B2B,Spread,Total,Player,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS_Hit_Pct,PTS_P_Diff,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST_Hit_Pct,AST_P_Diff,Off_REB,Off_L5_REB,AVG_REB_H2H,Def_REB,Def_L5_REB,REB_line,REB_Hit_Pct,REB_P_Diff,Off_PR,Off_L5_PR,AVG_PR_H2H,Def_PR,Def_L5_PR,PR_line,PR_Hit_Pct,PR_P_Diff,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA_Hit_Pct,PA_P_Diff,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA_Hit_Pct,RA_P_Diff,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA_Hit_Pct,PRA_P_Diff,Off_TPM,Off_L5_TPM,AVG_TPM_H2H,Def_TPM,Def_L5_TPM,TPM_line,TPM_Hit_Pct,TPM_P_Diff,Off_STL,Off_L5_STL,AVG_STL_H2H,Def_STL,Def_L5_STL,STL_line,STL_Hit_Pct,STL_P_Diff,Off_BLK,Off_L5_BLK,AVG_BLK_H2H,Def_BLK,Def_L5_BLK,BLK_line,BLK_Hit_Pct,BLK_P_Diff,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,Def_L5_STL_BLK,STL_BLK_line,STL_BLK_Hit_Pct,STL_BLK_P_Diff
0,2025-12-16,NYK,0,-2.5,233.5,Jalen Brunson,SAS,28.826087,32.8,NaN,21.476190,21.0,30.5,NaN,NaN,6.391304,6.0,NaN,4.952381,5.6,6.5,NaN,NaN,3.272727,2.6,NaN,5.000000,4.6,2.5,NaN,NaN,31.956522,35.4,NaN,26.476190,25.6,34.5,NaN,NaN,35.217391,38.8,NaN,26.428571,26.6,37.5,NaN,NaN,9.521739,8.6,NaN,9.952381,10.2,9.5,NaN,NaN,38.347826,41.4,NaN,31.428571,31.2,41.5,NaN,NaN,2.782609,3.2,NaN,2.523810,1.8,2.5,NaN,NaN,1.545455,1.4,NaN,1.523810,1.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-16,SAS,0,2.5,233.5,De'Aaron Fox,NYK,23.882353,22.4,NaN,17.947368,17.4,20.5,NaN,NaN,6.176471,5.6,NaN,6.315789,5.4,5.5,NaN,NaN,3.647059,4.0,NaN,4.631579,5.4,3.5,NaN,NaN,27.529412,26.4,NaN,22.578947,22.8,24.5,NaN,NaN,30.058824,28.0,NaN,24.263158,22.8,25.5,NaN,NaN,9.823529,9.6,NaN,10.947368,10.8,9.5,NaN,NaN,33.705882,32.0,NaN,28.894737,28.2,29.5,NaN,NaN,2.529412,2.8,NaN,2.052632,1.8,2.5,NaN,NaN,1.846154,1.8,NaN,0.894737,0.8,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-12-16,SAS,0,2.5,233.5,Stephon Castle,NYK,18.437500,17.4,NaN,17.947368,17.4,17.5,NaN,NaN,6.937500,4.8,NaN,6.315789,5.4,5.5,NaN,NaN,6.466667,8.0,NaN,4.631579,5.4,5.5,NaN,NaN,24.500000,23.4,NaN,22.578947,22.8,22.5,NaN,NaN,25.375000,22.2,NaN,24.263158,22.8,23.5,NaN,NaN,13.000000,10.8,NaN,10.947368,10.8,11.5,NaN,NaN,31.437500,28.2,NaN,28.894737,28.2,29.5,NaN,NaN,1.636364,1.6,NaN,2.052632,1.8,1.5,NaN,NaN,2.000000,1.6,NaN,0.894737,0.8,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-12-16,SAS,0,2.5,233.5,Devin Vassell,NYK,15.720000,15.8,12.000000,19.227273,17.0,13.5,NaN,NaN,2.909091,3.2,4.25,3.772727,3.4,1.5,NaN,NaN,3.760000,4.2,2.750000,3.818182,4.0,3.5,NaN,NaN,19.480000,20.0,14.75,23.045455,21.0,17.5,NaN,NaN,18.280000,19.0,16.25,23.000000,20.4,16.5,NaN,NaN,6.320000,7.4,7.00,7.590909,7.4,5.5,NaN,NaN,22.040000,23.2,19.00,26.818182,24.4,19.5,NaN,NaN,3.136364,3.2,1.25,2.954545,3.2,2.5,NaN,NaN,1.733333,1.6,0.25,1.318182,1.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-16,SAS,0,2.5,233.5,Dylan Harper,NYK,13.066667,12.4,NaN,19.227273,17.0,10.5,NaN,NaN,4.230769,4.2,NaN,3.772727,3.4,2.5,NaN,NaN,3.615385,3.8,NaN,3.818182,4.0,2.5,NaN,NaN,16.200000,16.2,NaN,23.045455,21.0,13.5,NaN,NaN,16.733333,16.6,NaN,23.000000,20.4,13.5,NaN,NaN,7.285714,8.0,NaN,7.590909,7.4,5.5,NaN,NaN,19.866667,20.4,NaN,26.818182,24.4,16.5,NaN,NaN,1.111111,1.0,NaN,2.954545,3.2,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2025-12-16,NYK,0,-2.5,233.5,Jordan Clarkson,SAS,10.000000,8.0,16.250000,18.720000,22.0,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2025-12-16,NYK,0,-2.5,233.5,Mikal Bridges,SAS,16.400000,13.8,20.

# REPOPULATE DATA

In [4]:
def pick_finder2(stat, season, now, df_inj, collect=False):    

    df_gms = pd.read_csv(f"../tables/{season}/nba_schedule.csv")
    df_gms['Date'] = pd.to_datetime(df_gms.Date)
    df_gms = df_gms[(df_gms.Date == now)]
    tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
    df_teams = pd.read_csv("../src/team_info_xref.csv")
    
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])

    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = {season} AND Pos != 'None' AND Date < '{now}'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = {season} AND Pos != 'None' AND Date < '{now}'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat}
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()
    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 18 AND Season = {season} AND Pos = '{pos}' AND Date < '{now}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                                 """).fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 18 AND Season = {season} AND Pos = '{pos}' AND Date < '{now}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date < '{now}'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
    
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [5]:
def repopulate_old_szns(season):
    schd = pd.read_csv(f"../tables/{season}/nba_schedule.csv")
    schd['Date'] = pd.to_datetime(schd.Date)
#     schd = schd[(schd.Date >= '2025-11-03')]
    df_lines = pd.read_csv(f"../tables/{season}/parlay_lines.csv")
    df_teams = pd.read_csv("../src/team_info_xref.csv")
    for date in schd.Date.unique().tolist():
        dt_str = date.strftime("%Y-%m-%d")
        print(dt_str)
        now = dt_str

        if now not in ['2023-05-02']:        
            df_inj = injury.get_reportdata(datetime.strptime(now, "%Y-%m-%d"), return_df=True)
            df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
            df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
            df_inj['Player'] = df_inj['Player'].str.strip()
            df_inj['Date'] = pd.to_datetime(df_inj['Date'])
            df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                                 JOIN df_teams ON df_inj.Team = df_teams.Team
                                 WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()
        else:
            df_inj = pd.DataFrame(columns=['Date', 'Team', 'Player', 'Status', 'Reason'])
        
        df_stats = pd.DataFrame()
        for stat in categories:
            print(f"==={stat}===")
            df_temp = pick_finder2(stat, season, now, df_inj, collect=True)
            df_stats = pd.concat([df_stats, df_temp], ignore_index=True)

        df_stats = con.execute("SELECT Date, Team, B2B, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk) FROM df_stats").fetchdf()
        df_save = pd.DataFrame()
        for player in df_stats.Player.unique():
            df_temp = df_stats[(df_stats.Player == player)]
            for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
                df_temp[col] = df_temp[col].ffill()
                df_temp[col] = df_temp[col].backfill()
            df_temp = df_temp.drop_duplicates()
            df_save = pd.concat([df_save, df_temp])
        print('Saving for Date:', now)
        print('row count - ', df_save.shape[0])
#         display(df_save)
        partition_save_df(df_save, f"../tables/{season}/parlay_stats.csv")